In [1]:
import tensorflow as tf

from tensorflow.keras.layers import LSTM

# from tensorflow.keras.models import Sequential

from tensorflow.keras.losses import MeanSquaredError
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
training_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\PortTraining_1404_002.csv"
#training_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\train1_port_new.csv"
test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\StbdTesting_1404_002.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\test1_stbd_new.csv"
prediction_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\AIOutput_CSV_Files\PredictionPatternTestPort.csv"

In [3]:
df_train=pd.read_csv(training_path)
df_test=pd.read_csv(test_path)
df_prediction=pd.read_csv(prediction_path)


In [4]:

#df_predictedDOA = df_prediction.iloc[:,4]
#df_prediction
df_predictedDOA = df_prediction[["DOAPrediction"]]
df_predictedDOA

,DOAPrediction
0,0.000000
1,-0.020950
2,-0.041899
3,-0.062849
4,-0.083799
...,...
5504452,-89.916201
5504453,-89.937151
5504454,-89.958101
5504455,-89.979050


In [5]:
df_reference = df_train[["DOA", "AngleUncertainty"]]
df_reference

,DOA,AngleUncertainty
0,-55.809999,0
1,-55.809999,0
2,-55.809999,0
3,-55.809999,0
4,-55.809999,0
...,...,...
5504452,-82.134998,206
5504453,-80.939998,97
5504454,-80.624998,32
5504455,-80.924998,48


In [6]:
df_reference['AngleUncertainty'].max()

255

In [7]:
df_reference['AngleUncertainty'].mean()

91.25883134340044

In [8]:
df_reference['AngleUncertainty'].var()

5990.4468260361145

In [9]:
df_comparison = df_predictedDOA.join(df_reference)
df_comparison

,DOAPrediction,DOA,AngleUncertainty
0,0.000000,-55.809999,0
1,-0.020950,-55.809999,0
2,-0.041899,-55.809999,0
3,-0.062849,-55.809999,0
4,-0.083799,-55.809999,0
...,...,...,...
5504452,-89.916201,-82.134998,206
5504453,-89.937151,-80.939998,97
5504454,-89.958101,-80.624998,32
5504455,-89.979050,-80.924998,48


In [10]:
theShape = df_comparison.shape
numRows= theShape[0]

In [11]:
numCols = 1
#5504457 
df_results = pd.DataFrame(index=range(numRows),columns=range(numCols))
#df_results = pd.DataFrame({'PassFail'})
#df_comparison.insert()
df_results.columns = {"PassFail"}
df_results

,PassFail
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
5504452,NaN
5504453,NaN
5504454,NaN
5504455,NaN


In [12]:
df_comparison = df_comparison.join(df_results)
df_comparison

,DOAPrediction,DOA,AngleUncertainty,PassFail
0,0.000000,-55.809999,0,NaN
1,-0.020950,-55.809999,0,NaN
2,-0.041899,-55.809999,0,NaN
3,-0.062849,-55.809999,0,NaN
4,-0.083799,-55.809999,0,NaN
...,...,...,...,...
5504452,-89.916201,-82.134998,206,NaN
5504453,-89.937151,-80.939998,97,NaN
5504454,-89.958101,-80.624998,32,NaN
5504455,-89.979050,-80.924998,48,NaN


In [13]:
df_comparison['Delta'] = df_comparison['DOAPrediction'] - df_comparison['DOA']
df_comparison['AbsDelta'] = df_comparison['Delta'].abs()
df_comparison

,DOAPrediction,DOA,AngleUncertainty,PassFail,Delta,AbsDelta
0,0.000000,-55.809999,0,NaN,55.809999,55.809999
1,-0.020950,-55.809999,0,NaN,55.789049,55.789049
2,-0.041899,-55.809999,0,NaN,55.768099,55.768099
3,-0.062849,-55.809999,0,NaN,55.747150,55.747150
4,-0.083799,-55.809999,0,NaN,55.726200,55.726200
...,...,...,...,...,...,...
5504452,-89.916201,-82.134998,206,NaN,-7.781203,7.781203
5504453,-89.937151,-80.939998,97,NaN,-8.997153,8.997153
5504454,-89.958101,-80.624998,32,NaN,-9.333102,9.333102
5504455,-89.979050,-80.924998,48,NaN,-9.054052,9.054052


In [14]:
# less than (0.02*AngleUncertainty)/2, since this is the 2sigma value, and we want all values to be under 1 sigma.
df_comparison['PassFail'] = df_comparison['AbsDelta'] <= (0.02*df_comparison['AngleUncertainty']/2)
df_comparison

,DOAPrediction,DOA,AngleUncertainty,PassFail,Delta,AbsDelta
0,0.000000,-55.809999,0,False,55.809999,55.809999
1,-0.020950,-55.809999,0,False,55.789049,55.789049
2,-0.041899,-55.809999,0,False,55.768099,55.768099
3,-0.062849,-55.809999,0,False,55.747150,55.747150
4,-0.083799,-55.809999,0,False,55.726200,55.726200
...,...,...,...,...,...,...
5504452,-89.916201,-82.134998,206,False,-7.781203,7.781203
5504453,-89.937151,-80.939998,97,False,-8.997153,8.997153
5504454,-89.958101,-80.624998,32,False,-9.333102,9.333102
5504455,-89.979050,-80.924998,48,False,-9.054052,9.054052


In [15]:
PassCount = df_comparison['PassFail'].sum()
PassCount

145620

In [16]:
FailCount = (~df_comparison['PassFail']).sum()
FailCount

5358837

In [17]:
TotalCount = PassCount+FailCount
TotalCount

5504457

In [18]:
PercentPass = 100*(PassCount/TotalCount)
PercentPass

2.645492552671408